# Introducción al Concepto de Regularización


En el contexto de la regresión, la regularización se refiere a técnicas para restringir/reducir las estimaciones de coeficientes hacia cero. 

La reducción de los coeficientes puede 1) mejorar el ajuste del modelo y 2) reducir la varianza de los coeficientes. 

Dos tipos comunes de regularización son ridge y lasso. Recuerde que la regresión lineal de mínimos cuadrados minimiza la suma residual de cuadrados (RSS). En otras palabras, minimiza

$ RSS = \displaystyle \sum^{n}_{i=1} (y_i - \beta_0 - \sum^{p}_{j=1} \beta_j x_{ij})^2 $

En ridge y lasso, agregamos un termino al valor que estamos intentando minimizar.

En ridge, minimizamos

$ RSS + \lambda \displaystyle \sum^{p}_{j=1} \beta_j^2 $

En lasso, minimizamos

$ RSS + \lambda \displaystyle \sum^{p}_{j=1} |\beta_j| $

El $\lambda$ (pronounciado "lambda") es un hiper-prámetro que determina que tan "fuerte" es el efecto de la regularización.

Nota: en oportunidades $\alpha$ (pronounciado "alpha") se usa como notación en vez de $\lambda$.

Una forma útil de usar la regresión de ridge o lasso es ejecutar la regresión en un rango de alfas y ver qué variables mantienen un coeficiente beta grande durante más tiempo. ¡Son estas variables las que tienen el mejor poder predictivo! 

Puede encontrar más información detallada aquí: [Regularization Regression](https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/)

In [ ]:
# Cargar paquetes de Python
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = (12, 8)
sns.set()
sns.set(font_scale=1.5)

# Paquetes para validar los supuestos
from scipy import stats as stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Paquetes para la regularización
from sklearn.linear_model import Lasso
from math import pow, sqrt

np.random.seed(1234)

2) Carga de Data
---

In [ ]:
path = '../data/'
filename = 'loans.csv'

df = pd.read_csv(path+filename)

In [ ]:
df.dtypes

In [ ]:
# Creamos variables indicadoras por pais (dummies)
for country in df['location_country_code'].unique():
    if country is not np.nan:
        df['country_'+country] = np.where(df.location_country_code == country, 1, 0)

In [ ]:
# Creamos variables indicadoras por sector (dummies)
for sect in df['sector'].unique():
    df['sector_'+sect] = np.where(df.sector == sect, 1, 0)

In [ ]:
df.dtypes

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# Definimos la variable dependiente
y = df['loan_amount']
# Definimos las variables independientes
X = df[['lender_count', 'sector_Education', 'sector_Clothing', 
        'sector_Personal Use', 'sector_Retail', 'sector_Transportation', 'sector_Agriculture']]
# Agregamos el intercepto a la variable independiente
X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model1.summary())

In [ ]:
alphas = np.arange(0.001, 0.502, 0.002)
lasso_coefs = []
X_train_lasso= X_train[X_train.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_train_lasso, y_train)
    lasso_coefs.append(lassoreg.coef_)

In [ ]:
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_train_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

Retail and Transportation go to 0 when alpha is 0.3.  Let's try removing these from the model.

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# Define the dependent variable
y = df['loan_amount']
# Define the independent variables
X = df[['lender_count', 'sector_Education', 'sector_Clothing', 
        'sector_Personal Use', 'sector_Agriculture']]
# Add an intercept term to the independent variables
X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model2 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model2.summary())

Aunque eliminamos dos variables dependientes del análisis, nuestro R cuadrado y el R cuadrado ajustado se mantuvieron igual. Esto significa que las dos variables que eliminamos (Transportation y Retail) son menos importantes para el monto del préstamo. El ejemplo anterior muestra cómo podemos usar la regularización para la selección de variables.

# Aspectos importante de la regularización

Recuerde que con regresión lineal de mínimos cuadrados, los coeficientes son equivalentes de escala. En otras palabras, multiplicar una variable por una constante $ c $ es equivalente a escalar de la estimación del coeficiente de mínimos cuadrados por un factor de 1 / $ c $. 

Demostremos este hecho creando un conjunto de datos de ejemplo que tenga tres variables: 1) cantidad de dinero que se gana en un restaurante en un día, 2) distancia en metros a la universidad más cercana, 3) distancia en kilómetros al hospital más cercano.

In [ ]:
np.random.seed(1234)

earnings = np.random.normal(2000, 300, 50)
university_distances = np.random.normal(7000,2000,50)
hospital_distances = np.random.normal(7,2,50)

earnings = [a if a > 0 else -a for a in earnings]
university_distances = [a if a > 0 else -a for a in university_distances]
hospital_distances = [a if a > 0 else -a for a in hospital_distances]

In [ ]:
df = pd.DataFrame({"earnings": sorted(earnings), "university": sorted(university_distances, reverse=True), 
                   'hospital' : sorted(hospital_distances, reverse=True)})

In [ ]:
df

In [ ]:
# Grafico: distancia a la universidad mas cercana (en metros) vs. ganancia

ax = sns.regplot(x='earnings', y='university', data=df, fit_reg=False)
ax.set_title('Scatter plot of distance to nearest university (in meters) vs earnings')

In [ ]:
# Grafico: distancia al hospital mas cercano (en metros) vs. ganancia

ax = sns.regplot(x='earnings', y='hospital', data=df, fit_reg=False)
ax.set_title('Scatter plot of distance to nearest hospital (in kilometers) vs earnings')

Realicemos una regresión lineal multivariante sin escalar ninguna variable y comparemos los resultados con un modelo en el que estandaricemos las variables de distancia para que ambos usen kilómetros.

In [ ]:
model1 = smf.ols(formula = 'earnings ~ university + hospital', data = df).fit()
print(model1.summary())

El R cuadrado es 0.938 y el R cuadrado ajustado es 0.935. Los coeficientes para el intercepto, la universidad y el hospital son 3024.1009, -0.0643 y -76.3083. Ahora escalemos la variable universitaria para que esté en kilómetros en lugar de metros.

In [ ]:
df_scaled = df.copy()
df_scaled['university'] = df_scaled['university']/1000

df_scaled

In [ ]:
model2 = smf.ols(formula = 'earnings ~ university + hospital', data = df_scaled).fit()
print(model2.summary())

El R cuadrado es 0.938 y el R cuadrado ajustado es 0.935. Los coeficientes para el intercepto, la universidad y el hospital son 3024.1009, -64.3473 y -76.3083. Ahora, cambiamos la variable universitaria al escalarla por una constante y el coeficiente resultante fue escalado por la misma constante. Los p-values no cambiaron y los coeficientes de las otras variables no cambiaron. 

¿Qué crees que hará el escalado si incorporamos la regularización mediante el uso de regresión lasso ridge? ¿Crees que la escala tendrá un efecto en los coeficientes de las variables?

<br>
<br>
<br>
<br>

Ejecutemos la regresión lasso en nuestros datos sin escala y con los datos escalados y veamos qué sucede.

# Data sin escalar

In [ ]:
X = df[['university', 'hospital']]
y = df['earnings']

alphas = np.arange(0.001, 1, 0.002)
lasso_coefs = []
X_lasso= X[X.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_lasso, y)
    lasso_coefs.append(lassoreg.coef_)
    
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

El gráfico anterior muestra los coeficientes para las variables de universidad y hospital en 0 y aproximadamente -75, respectivamente. ¿Conservaría o eliminaría estas variables de su modelo? ¿Por qué?

<br>
<br>
<br>
<br>

# Data escalada

In [ ]:
X = df_scaled[['university', 'hospital']]
y = df_scaled['earnings']

alphas = np.arange(0.001, 1, 0.002)
lasso_coefs = []
X_lasso= X[X.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_lasso, y)
    lasso_coefs.append(lassoreg.coef_)
    
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

El gráfico anterior muestra que el coeficiente para las variables de la universidad y el hospital es de alrededor de -64 y -76, respectivamente. ¿Conservaría o eliminaría estas variables de su modelo? ¿Por qué?

<br>
<br>
<br>
<br>

Claramente, la escala afecta los coeficientes y, por lo tanto, afecta los resultados de la regresión lasso. Por lo tanto, es mejor aplicar técnicas de regularización como ridge y lasso después de estandarizar los predictores. Puede estandarizar los predictores aplicando la siguiente fórmula:

$ \tilde{x}_{ij} = \frac{x_{ij}}{\sqrt{\frac{1}{n} \sum_{i=1}^{n} (x_{ij} - \bar{x}_{j})^2}} $

Así que ahora tomemos los datos sin escala y creemos un nuevo conjunto de datos donde estandaricemos los predictores.

In [ ]:
df_standardized = df.copy()

In [ ]:
university_mean = df_standardized['university'].mean()
university_denom = sqrt(sum((df_standardized['university']-university_mean)**2)/len(df_standardized['university']))

hospital_mean = df_standardized['hospital'].mean()
hospital_denom = sqrt(sum((df_standardized['hospital']-hospital_mean)**2)/len(df_standardized['hospital']))

In [ ]:
df_standardized['university'] = df_standardized['university']/university_denom
df_standardized['hospital'] = df_standardized['hospital']/hospital_denom


In [ ]:
df_standardized

In [ ]:
X = df_standardized[['university', 'hospital']]
y = df_standardized['earnings']

alphas = np.arange(0.001, 1, 0.002)
lasso_coefs = []
X_lasso= X[X.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_lasso, y)
    lasso_coefs.append(lassoreg.coef_)
    
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

Ahora que hemos escalado nuestras variables, ¡los coeficientes han vuelto a estar dentro del mismo orden de magnitud! 

Recuerde siempre estandarizar las variables cuando use la regularización.